In [106]:
import pandas as pd
import numpy as np
import sys
import os

# load functions
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from lib.functions_prepare_data import *

## Hospital details

### Data In

In [162]:
# load details data
df_hospital_details = pd.read_csv('../data/in/staging/atlas_details.csv', dtype={'hospital_id': str, 'nursing_count': str})
df_hospital_details.head()

,hospital_id,total_treatments,total_treatments_label,nursing_quotient,nursing_quotient_label,nursing_count,provider_type,bed_count,semi_residential_count,emergency_service
0,771003,23479,sehr viele,60.11,weit unterdurchschnittlich,363,öffentlich,539,52,Stufe 2 - Erweiterte Notfallversorgung
1,771011,13563,viele,55.68,unterdurchschnittlich,227,öffentlich,320,20,Stufe 2 - Erweiterte Notfallversorgung
2,771012,15347,viele,48.69,überdurchschnittlich,268,öffentlich,300,8,Stufe 2 - Erweiterte Notfallversorgung
3,771015,1422,sehr wenige,39.15,weit überdurchschnittlich,40,privat,59,10,"Ja, die Notfallstufe ist noch nicht vereinbart."
4,771016,21996,sehr viele,46.59,überdurchschnittlich,352,privat,508,8,Stufe 2 - Erweiterte Notfallversorgung


### Processing

In [163]:
# drop total_treatments_label and nursing_count_label column
df_hospital_details = df_hospital_details.drop(columns=['nursing_quotient_label', 'total_treatments_label'])

In [164]:
# correct dtype of nursing_count
df_hospital_details['nursing_count'] = df_hospital_details['nursing_count'].str.replace('.', '')
df_hospital_details['nursing_count'] = df_hospital_details['nursing_count'].astype(int)

# clean up the emergency_service column
df_hospital_details['emergency_service'] = df_hospital_details['emergency_service'].apply(lambda x: x.split('\n')[0])

# create new column has_emergency_service
has_emergency_service_mapping = {
    'Stufe 1 - Basisnotfallversorgung': True,
    'Stufe 2 - Erweiterte Notfallversorgung': True,
    'Stufe 3 - Umfassende Notfallversorgung': True,
    'Ja, die Notfallstufe ist noch nicht vereinbart.': True,
    'Keine Teilnahme an einer Notfallstufe.': False,
    'Keine Information verfügbar.': False
}
df_hospital_details['has_emergency_service'] = df_hospital_details['emergency_service'].map(has_emergency_service_mapping)

# rename column emergency_service to emergency_service_level
df_hospital_details = df_hospital_details.rename(columns={'emergency_service': 'emergency_service_level'})

In [165]:
# create mapping for emergency_service_level
emergency_service_level_mapping = {
    'Stufe 1 - Basisnotfallversorgung': 1,
    'Stufe 2 - Erweiterte Notfallversorgung': 2,
    'Stufe 3 - Umfassende Notfallversorgung': 3,
    'Ja, die Notfallstufe ist noch nicht vereinbart.': 0,
    'Keine Teilnahme an einer Notfallstufe.': -1,
    'Keine Information verfügbar.': -2
}

# map emergency_service_level
df_hospital_details['emergency_service_level'] = df_hospital_details['emergency_service_level'].map(emergency_service_level_mapping)

In [166]:
# create mapping for provider_type
df_hospital_details['provider_type'].value_counts()
provider_type_mapping = {
    'freigemeinnützig': 'F',
    'öffentlich': 'O',
    'privat': 'P'
}

# map provider_type
df_hospital_details['provider_type'] = df_hospital_details['provider_type'].map(provider_type_mapping)
df_hospital_details.rename(columns={'provider_type': 'provider_type_code'})

# create code dictionary for database
df_provider_type_codes = pd.DataFrame({'provider_type_code': ['F', 'O', 'P']})
df_provider_type_dict_de = pd.DataFrame({'provider_type_code': ['F', 'O', 'P'], 'provider_type': ['freigemeinnützig', 'öffentlich', 'privat'], 'language_code': ['de', 'de', 'de']})
df_provider_type_dict_en = pd.DataFrame({'provider_type_code': ['F', 'O', 'P'], 'provider_type': ['non-profit', 'public', 'private'], 'language_code': ['en', 'en', 'en']})
df_privider_type_dict = pd.concat([df_provider_type_dict_de, df_provider_type_dict_en], axis=0)

# renaming prvoider columns
df_hospital_details = df_hospital_details.rename(columns={'provider_type': 'provider_type_code'})
df_provider_type_dict = df_privider_type_dict.rename(columns={'provider_type': 'provider_type_name'})

In [167]:
# create additional columns to store total number of stations (bed + semi-residential)
df_hospital_details['total_stations_count'] = df_hospital_details['bed_count'] + df_hospital_details['semi_residential_count']

### Data Out

In [168]:
# save details data ready for database
df_hospital_details.to_csv('../data/db_csv/hospital_details.csv', index=False, encoding='utf-8')

# save provider type code dictionary and PK table ready for database
df_privider_type_dict.to_csv('../data/db_csv/provider_types_dict.csv', index=False, encoding='utf-8')
df_provider_type_codes.to_csv('../data/db_csv/provider_type_codes.csv', index=False, encoding='utf-8')

In [169]:
df_hospital_details.head()

,hospital_id,total_treatments,nursing_quotient,nursing_count,provider_type_code,bed_count,semi_residential_count,emergency_service_level,has_emergency_service,total_stations_count
0,771003,23479,60.11,363,O,539,52,2,True,591
1,771011,13563,55.68,227,O,320,20,2,True,340
2,771012,15347,48.69,268,O,300,8,2,True,308
3,771015,1422,39.15,40,P,59,10,0,True,69
4,771016,21996,46.59,352,P,508,8,2,True,516


In [170]:
df_privider_type_dict.head()

,provider_type_code,provider_type,language_code
0,F,freigemeinnützig,de
1,O,öffentlich,de
2,P,privat,de
0,F,non-profit,en
1,O,public,en


In [171]:
df_provider_type_codes.head()

,provider_type_code
0,F
1,O
2,P


## Hospital certificates

### Data In

In [116]:
df_hospital_certificates = pd.read_csv('../data/in/staging/atlas_certificates.csv')
df_hospital_certificates.head()

,hospital_id,certificate
0,771003,EndoProthetikZentrum der Maximalversorgung
1,771003,Onkologisches Zentrum
2,771003,Brustkrebszentrum
3,771003,Darmkrebszentrum
4,771003,Zentrum für Hämatologische Neoplasien


### Processing

In [117]:
# get certificate translation mapping
certificate_translation_mapping = get_certificate_translation_mapping()

In [118]:
# translate certificates
df_hospital_certificates_de = df_hospital_certificates.copy()
df_hospital_certificates_de['language_code'] = 'de'
df_hospital_certificates_en = df_hospital_certificates_de.copy()
df_hospital_certificates_en['certificate'] = df_hospital_certificates_en['certificate'].map(certificate_translation_mapping)
df_hospital_certificates_en['language_code'] = 'en'

In [119]:
# concatenate en and de dataframes
df_hospital_certificates = pd.concat([df_hospital_certificates_de, df_hospital_certificates_en], axis=0)

### Data Out

In [120]:
# save certificates data ready for db
df_hospital_certificates.to_csv('../data/db_csv/hospital_certificates.csv', index=False, encoding='utf-8')

In [121]:
df_hospital_certificates.head()

,hospital_id,certificate,language_code
0,771003,EndoProthetikZentrum der Maximalversorgung,de
1,771003,Onkologisches Zentrum,de
2,771003,Brustkrebszentrum,de
3,771003,Darmkrebszentrum,de
4,771003,Zentrum für Hämatologische Neoplasien,de


## Hospital departments

### Data In

In [122]:
# load department labels and hospital department data
df_department_labels = pd.read_csv('../data/in/staging/departments_general_dict.csv', dtype={'department_id': str})
df_department_focus_labels = pd.read_csv('../data/in/staging/departments_focus_dict.csv', dtype={'department_id': str})
df_department_labels_concat = pd.concat([df_department_labels, df_department_focus_labels])
df_hospital_departments = pd.read_csv('../data/in/staging/atlas_departments.csv')

### Processing

In [123]:
# rename department_labeles with 'ohne Differenzierung' (="no focus specified")
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].apply(lambda x: x.split('/')[0] if 'ohne Differenzierung' in x else x)

# streamline department_labels with '/ ' instead of '/'
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].apply(lambda x: x.replace('/ ', '/').replace(' /', '/'))

# correct '(Intensivabteilung)' to 'Intensivmedizin'
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].apply(lambda x: x.replace(' (Intensivabteilung)', '/Intensivmedizin'))

# rename department_label 'Neurologie/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)'
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].replace('Neurologie/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)', 'Neurologie/Schwerpunkt Schlaganfallpatienten')

# rename department_label 'Innere Medizin/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)'
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].replace('Innere Medizin/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)', 'Innere Medizin/Schwerpunkt Schlaganfallpatienten')

# rename department_label 'Orthopädie/Rheumatologie'
df_hospital_departments['department_name'] = df_hospital_departments['department_name'].replace('Orthopädie/Rheumatologie', 'Orthopädie/Schwerpunkt Rheumatologie')

In [124]:
# create dictionary to translate department labels to department ids
departments_dict = {}
for department_label in df_department_labels_concat['department_label']:
    departments_dict.update({department_label: df_department_labels_concat[df_department_labels_concat['department_label'] == department_label]['department_id'].values[0]})

# apply dictionary to rename columns
def map_values(value):
    return departments_dict.get(value, value)

df_hospital_departments['department_id'] = df_hospital_departments['department_name'].apply(map_values)

# remove department name column
df_hospital_departments = df_hospital_departments.drop(columns=['department_name'])

# rename column department_count to treatment_count for database
df_hospital_departments = df_hospital_departments.rename(columns={'department_count': 'treatment_count'})

# rename column department_label to department_name for database
df_department_labels_concat = df_department_labels_concat.rename(columns={'department_label': 'department_name'})

In [125]:
# get parent department reference for (sub)departments
df_department_labels_concat['parent_department_id'] = df_department_labels_concat['department_id'].apply(lambda x: x[0:2] + ('00'))
df_parent_department_labels_ = df_department_labels_concat[['department_id', 'department_name']].rename(columns={'department_id': 'parent_department_id','department_name': 'parent_department_name'})
df_departments = pd.merge(df_department_labels_concat, df_parent_department_labels_, on='parent_department_id', how='left')

In [126]:
# get department translation mapping
department_translation_mapping = get_department_translation_mapping()

In [127]:
# translate department names
df_departments_de = df_departments.copy()
df_departments_de['language_code'] = 'de'
df_departments_en = df_departments_de.copy()
df_departments_en['department_name'] = df_departments_en['department_name'].map(department_translation_mapping)
df_departments_en['parent_department_name'] = df_departments_en['parent_department_name'].map(department_translation_mapping)
df_departments_en['language_code'] = 'en'

In [128]:
# concatenate en and de dataframes
df_departments_dict = pd.concat([df_departments_de, df_departments_en], axis=0)

In [129]:
# create department codes for database PKs
df_department_ids = df_departments_dict[['department_id']].drop_duplicates()

In [130]:
# rename department_id to department_code
df_hospital_departments = df_hospital_departments.rename(columns={'department_id': 'department_code'})
df_departments_dict = df_departments_dict.rename(columns={'department_id': 'department_code', 'parent_department_id': 'parent_department_code'})
df_department_codes = df_department_ids.rename(columns={'department_id': 'department_code'})


### Data Out

In [131]:
# save data ready for database import
df_hospital_departments.to_csv('../data/db_csv/hospital_departments.csv', index=False, encoding='utf-8')
df_departments_dict.to_csv('../data/db_csv/departments_dict.csv', index=False, encoding='utf-8')
df_department_codes.to_csv('../data/db_csv/department_codes.csv', index=False, encoding='utf-8')

In [132]:
df_hospital_departments.head(3)

,hospital_id,treatment_count,department_code
0,771003,7598,0100
1,771003,6347,1500
2,771003,5825,2400


In [133]:
df_departments_dict.head(3)

,department_code,department_name,parent_department_code,parent_department_name,language_code
0,0100,Innere Medizin,0100,Innere Medizin,de
1,0200,Geriatrie,0200,Geriatrie,de
2,0300,Kardiologie,0300,Kardiologie,de


In [134]:
df_department_codes.head(3)

,department_code
0,0100
1,0200
2,0300


## Hospital treatments

### Data In

In [135]:
# load treatments data
df_hospital_treatments = pd.DataFrame()
m = 50
for k in range(34):
    df_hospital_treatments_ = pd.read_csv(f'../data/in/staging/treatments_chunks/atlas_treatments_sample_{k*m}-{k*m+m-1}.csv', dtype={'hospital_id': str})
    df_hospital_treatments = pd.concat([df_hospital_treatments, df_hospital_treatments_])

In [136]:
# load treatments dict data
df_treatments_dict = pd.read_csv('../data/in/staging/treatments_dict.csv', dtype={'treatment_id': str})

In [137]:
# create code table for database PKs
df_treatment_codes = df_treatments_dict[['treatment_code']]

In [138]:
treatment_translation_list = get_treatment_translation_list()

In [139]:
df_treatments_dict_de = df_treatments_dict.copy()
df_treatments_dict_de['language_code'] = 'de'

df_treatments_dict_en = df_treatments_dict_de.copy()
df_treatments_dict_en['treatment_name'] = treatment_translation_list
df_treatments_dict_en['language_code'] = 'en'

df_treatments_dict = pd.concat([df_treatments_dict_de, df_treatments_dict_en], axis=0)

### Processing

In [140]:
# rename columns for database
df_hospital_treatments = df_hospital_treatments.rename(columns={'count_number': 'treatment_count', 'count_label': 'treatment_count_label'})

# drop treatment_count_label column
df_hospital_treatments = df_hospital_treatments.drop(columns=['treatment_count_label'])

### Data Out

In [141]:
# save data ready for database import
df_treatments_dict.to_csv('../data/db_csv/treatments_dict.csv', index=False, encoding='utf-8')
df_hospital_treatments.to_csv('../data/db_csv/hospital_treatments.csv', index=False, encoding='utf-8')
df_treatment_codes.to_csv('../data/db_csv/treatment_codes.csv', index=False, encoding='utf-8')

In [142]:
df_treatments_dict.head(3)

,treatment_code,treatment_name,language_code
0,KAHKO0,Chirurgischer Herzklappenersatz,de
1,KAHM0,Minimal-invasiver Herzklappenersatz,de
2,KAKIA,Bypassoperation des Herzens,de


In [143]:
df_hospital_treatments.head(3)

,hospital_id,treatment_code,treatment_count
0,771003,KAHKO0,0
1,771003,KAHM0,0
2,771003,KAKIA,0


In [144]:
df_treatment_codes.head(3)

,treatment_code
0,KAHKO0
1,KAHM0
2,KAKIA


## Federal states

### Data In

In [145]:
# load zip dict data
df_zip_dict = pd.read_csv('../data/in/raw/zip_dict/georef-germany-postleitzahl.csv', dtype={'Name': str}, sep=';')[['Name', 'Land name']]
df_zip_dict = df_zip_dict.rename(columns={'Name': 'zip', 'Land name': 'federal_state_name'})

In [146]:
# load and prepare population data
df_population = pd.read_csv('../data/in/raw/federal_states/Bevölkerung - Bundesländer.csv', dtype={'Gesamt': str})[['Bundesland', 'Gesamt']]
df_population = df_population.rename(columns={'Bundesland': 'federal_state_name', 'Gesamt': 'population'})
df_population['population'] = df_population['population'].apply(lambda x: x.replace(' ', '')).astype(int)

# load and prepare area data
df_area = pd.read_csv('../data/in/raw/federal_states/Gebietsfläche.csv', dtype={'Gebietsfläche': str})[['Bundesland', 'Gebietsfläche']]
df_area = df_area.rename(columns={'Bundesland': 'federal_state_name', 'Gebietsfläche': 'area'})
df_area['area'] = df_area['area'].replace(',', '.', regex=True).astype(float)

### Processing

In [147]:
# translate state codes
federal_state_codes = {
    'Baden-Württemberg': 'BW',
    'Bayern': 'BY',
    'Berlin': 'BE',
    'Brandenburg': 'BB',
    'Bremen': 'HB',
    'Hamburg': 'HH',
    'Hessen': 'HE',
    'Niedersachsen': 'NI',
    'Mecklenburg-Vorpommern': 'MV',
    'Nordrhein-Westfalen': 'NW',
    'Rheinland-Pfalz': 'RP',
    'Saarland': 'SL',
    'Sachsen': 'SN',
    'Sachsen-Anhalt': 'ST',
    'Schleswig-Holstein': 'SH',
    'Thüringen': 'TH'
}

df_zip_dict['federal_state_code'] = df_zip_dict['federal_state_name'].map(federal_state_codes)

In [148]:
# create federal state dataframe
df_federal_states = pd.DataFrame({'federal_state_code': list(federal_state_codes.values()), 'federal_state_name': list(federal_state_codes.keys())})

In [149]:
# merge federal state  with area data
df_federal_states = pd.merge(df_federal_states, df_area, on='federal_state_name')

# merge federal states with population data
df_federal_states = pd.merge(df_federal_states, df_population, on='federal_state_name')

In [150]:
# get federal state translation list
federal_states_translation_list = get_federal_states_translation_list()

In [151]:
# translate federal states and create dict for database
df_federal_states_dict_ = df_federal_states.copy()
df_federal_states_dict_ = df_federal_states_dict_.drop(columns=['area', 'population'])

df_federal_states_dict_de = df_federal_states_dict_.copy()
df_federal_states_dict_de['language_code'] = 'de'

df_federal_states_dict_en = df_federal_states_dict_de.copy()
df_federal_states_dict_en['federal_state_name'] = federal_states_translation_list
df_federal_states_dict_en['language_code'] = 'en'

df_federal_states_dict = pd.concat([df_federal_states_dict_de, df_federal_states_dict_en], axis=0)

In [152]:
# drop state name in federal_states dataframe
df_federal_states = df_federal_states.drop(columns=['federal_state_name'])

### Data Out

In [153]:
# save data ready for database import
df_federal_states.to_csv('../data/db_csv/federal_states.csv', index=False, encoding='utf-8')
df_federal_states_dict.to_csv('../data/db_csv/federal_states_dict.csv', index=False, encoding='utf-8')

In [154]:
df_federal_states.head(3)

,federal_state_code,area,population
0,BW,35747.85,11280257
1,BY,70541.58,13369393
2,BE,891.12,3755251


In [155]:
df_federal_states_dict.head(3)

,federal_state_code,federal_state_name,language_code
0,BW,Baden-Württemberg,de
1,BY,Bayern,de
2,BE,Berlin,de


## Places and Hospital locations

### Data In

In [156]:
# load places data
df_places = pd.read_json('../data/in/raw/atlas/german-places.json', dtype={'p': str})
df_places = df_places.rename(columns={'p': 'zip', 'c': 'city_district', 'm': 'name', 'd': 'rural_district', 'lt': 'latitude', 'ln': 'longitude'}).drop(columns=['ct'])

# load hospital_locations data
df_hospital_locations = pd.read_csv('../data/in/staging/hospital_locations.csv', dtype={'hospital_id': str, 'zip': str})
df_hospital_locations = df_hospital_locations.drop(columns=['beds_number'])

### Processing

In [157]:
# get federal state code for each hospital location
df_hospital_locations['federal_state_code'] = df_hospital_locations['zip'].apply(lambda x: df_zip_dict[df_zip_dict['zip'] == x]['federal_state_code'].values[0] if len(df_zip_dict[df_zip_dict['zip'] == x]['federal_state_code'].values) > 0 else np.nan)

# get federal state code for missing zips in df_zip_dict for hospital_locations
missing_zips_dict_locations = {
    '99437': 'TH',
    '19049': 'MV',
}

df_hospital_locations['federal_state_code'] = df_hospital_locations['federal_state_code'].fillna(df_hospital_locations['zip'].map(missing_zips_dict_locations))

In [158]:
# get federal state code for all places
df_places['federal_state_code'] = df_places['zip'].apply(lambda x: df_zip_dict[df_zip_dict['zip'] == x]['federal_state_code'].values[0] if len(df_zip_dict[df_zip_dict['zip'] == x]['federal_state_code'].values) > 0 else np.nan)

# get federal state code for missing zips in df_zip_dict for places
missing_zips_dict_places = {
    '60312': 'HE',
    '60315': 'HE',
    '64743': 'HE',
    '81248': 'BY',
    '99331': 'TH'
}

df_places['federal_state_code'] = df_places['federal_state_code'].fillna(df_places['zip'].map(missing_zips_dict_places))

### Data Out

In [159]:
# save data ready for database import
df_hospital_locations.to_csv('../data/db_csv/hospital_locations.csv', index=False, encoding='utf-8')
df_places.to_csv('../data/db_csv/places.csv', index=False, encoding='utf-8')

In [160]:
df_hospital_locations.head(3)

,name,street,city,zip,phone,mail,latitude,longitude,link,hospital_id,federal_state_code
0,Klinikum Südstadt Rostock,Südring 81,Rostock,18059,+49 (0)381/4401-0,info@kliniksued-rostock.de,54.071751,12.107268,https://bundes-klinik-atlas.de/krankenhaussuch...,771003,MV
1,Harzklinikum Dorothea Christiane Erxleben GmbH...,Ditfurter Weg 24,Quedlinburg,06484,+49 (0)3946/9090,info@harzklinikum.com,51.795329,11.163533,https://bundes-klinik-atlas.de/krankenhaussuch...,771011,ST
2,Harzklinikum Dorothea Christiane Erxleben GmbH...,Ilsenburger Straße 15,Wernigerode,38855,+49 (0)3946/9090,info@harzklinikum.com,51.835429,10.774409,https://bundes-klinik-atlas.de/krankenhaussuch...,771012,ST


In [161]:
df_places.head(3)

,zip,city_district,name,rural_district,latitude,longitude,federal_state_code
0,24937,Altstadt,Flensburg,Flensburg,54.78643,9.43339,SH
1,24939,Altstadt,Flensburg,Flensburg,54.78643,9.43339,SH
2,24943,Engelsby,Flensburg,Flensburg,54.79203,9.47724,SH
